In [53]:
import pandas as pd
import numpy as np
contest=pd.read_csv('WPX_CW_2023.csv')

In [69]:
# Easy Markdown formatting
def df_to_md(pd):
    text=pd.to_markdown();
    for a in text.split('\n'):
        print(f"{a}")

from datetime import timedelta, datetime

def ceil_date(date, **kwargs):
    secs = timedelta(**kwargs).total_seconds()
    return datetime.fromtimestamp(date.timestamp() + secs - date.timestamp() % secs)

def floor_date(date, **kwargs):
    secs = timedelta(**kwargs).total_seconds()
    return datetime.fromtimestamp(date.timestamp() - date.timestamp() % secs)

contest['date']=pd.to_datetime(contest.date,format='%Y-%m-%d %H:%M:%S',errors="ignore")     
contest['date_rounded_5m'] =contest['date'].apply(lambda x: floor_date(x, minutes=5))      
contest=contest.dropna()

print(f"Total records {len(contest)}")

Total records 3900762


In [70]:
# Lets just see the 10M band
TenM = contest[contest.band=='10m']
DU_Spots_All = contest[contest.dx_pfx=='DU']
DU_Spots_10M = contest[(contest.dx_pfx=='DU')&(contest.band=='10m')]
print(f"We have {len(DU_Spots_10M)} spots for DU calls on 10m band.")


We have 3417 spots for DU calls on 10m band.


In [71]:
# Lets Summarize 10M DU Spots 
du_10m_summary=DU_Spots_10M.groupby(['dx'])['callsign'].count().sort_values(ascending=False).reset_index()
du_10m_summary.columns=['Call','Spotted_Count']

In [72]:
df_to_md(du_10m_summary[du_10m_summary.Spotted_Count>10])

|    | Call   |   Spotted_Count |
|---:|:-------|----------------:|
|  0 | 4F3OM  |            1009 |
|  1 | 4I1EBC |             789 |
|  2 | DU3T   |             500 |
|  3 | DV3A   |             488 |
|  4 | DU1WBX |             256 |
|  5 | 4D3X   |             188 |
|  6 | 4F3BZ  |              99 |
|  7 | DU1EV  |              31 |
|  8 | DU1VGX |              16 |
|  9 | DX9EVM |              13 |


In [77]:
DU_QRP = contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='4E1ABC') |(contest.dx=='4F3OM'))]
DU_HP = contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='DU3T') |(contest.dx=='DV3A')|(contest.dx=='DU1EV'))]
DU_LP=contest[(contest.dx_pfx=='DU')&(contest.band=='10m') & ((contest.dx=='4D3X') |(contest.dx=='4E1AGW')|(contest.dx=='4F3BZ'))]

In [87]:
HP_vs_QRP=pd.merge(DU_HP,DU_QRP,on=['callsign','date_rounded_5m'],how='inner')[['date_rounded_5m','callsign','freq_x','dx_x','freq_y','dx_y','db_x','db_y']]
df_to_md(HP_vs_QRP[HP_vs_QRP.db_y >= HP_vs_QRP.db_x])

|     | date_rounded_5m     | callsign   |   freq_x | dx_x   |   freq_y | dx_y   |   db_x |   db_y |
|----:|:--------------------|:-----------|---------:|:-------|---------:|:-------|-------:|-------:|
|  16 | 2022-05-28 08:50:00 | VK6ANC     |  28037.6 | DV3A   |  28019.9 | 4F3OM  |     22 |     25 |
|  36 | 2022-05-28 10:30:00 | VU3KAZ     |  28024.4 | DU3T   |  28032   | 4F3OM  |     21 |     21 |
|  48 | 2022-05-28 10:50:00 | BG4WOM     |  28024.7 | DU3T   |  28032.2 | 4F3OM  |     25 |     31 |
|  49 | 2022-05-28 10:50:00 | BG4WOM     |  28053.5 | DV3A   |  28032.2 | 4F3OM  |     24 |     31 |
|  61 | 2022-05-28 11:00:00 | BD7LAE     |  28024.4 | DU3T   |  28039.9 | 4F3OM  |     32 |     32 |
| 141 | 2022-05-30 06:30:00 | VK4CT      |  28002.1 | DV3A   |  28032.9 | 4F3OM  |     13 |     14 |


In [90]:
LP_vs_QRP=pd.merge(DU_LP,DU_QRP,on=['callsign','date_rounded_5m'],how='inner')[['date_rounded_5m','callsign','freq_x','dx_x','freq_y','dx_y','db_x','db_y']]
df_to_md(LP_vs_QRP[LP_vs_QRP.db_y >= LP_vs_QRP.db_x])

|    | date_rounded_5m     | callsign   |   freq_x | dx_x   |   freq_y | dx_y   |   db_x |   db_y |
|---:|:--------------------|:-----------|---------:|:-------|---------:|:-------|-------:|-------:|
| 30 | 2022-05-29 14:00:00 | JH7CSU1    |  28009.2 | 4F3BZ  |  28040.1 | 4F3OM  |     12 |     21 |
| 34 | 2022-05-29 14:30:00 | JA4ZRK     |  28033   | 4F3BZ  |  28001   | 4F3OM  |      3 |      8 |
| 46 | 2022-05-29 16:05:00 | JH7CSU1    |  28034.1 | 4F3BZ  |  28023.6 | 4F3OM  |     18 |     20 |
| 47 | 2022-05-29 16:10:00 | VU3KAZ     |  28034   | 4F3BZ  |  28023.5 | 4F3OM  |     15 |     17 |


In [135]:

TenMBeacon=TenM[TenM['mode']=='NCDXF B']
len(TenMBeacon)

438

In [132]:
pd.merge(TenMBeacon,DU_QRP,on=['callsign'],how='inner')['callsign'].unique()

array(['JI1HFJ', 'HS/F8UKP', 'VU3KAZ'], dtype=object)

In [133]:
pd.merge(TenMBeacon,DU_LP,on=['callsign'],how='inner')['callsign'].unique()

array(['JI1HFJ', 'HS/F8UKP', 'VU3KAZ', '9M2CNC'], dtype=object)

In [150]:
df_to_md(pd.merge(TenMBeacon,DU_QRP,on=['callsign'],how='inner')[['callsign','dx_x','db_x']].groupby(['callsign','dx_x']).max().reset_index())

|    | callsign   | dx_x   |   db_x |
|---:|:-----------|:-------|-------:|
|  0 | HS/F8UKP   | 4X6TU  |      6 |
|  1 | HS/F8UKP   | JA2IGY |     13 |
|  2 | HS/F8UKP   | RR9O   |      6 |
|  3 | HS/F8UKP   | VK6RBP |     16 |
|  4 | HS/F8UKP   | VR2B   |     16 |
|  5 | JI1HFJ     | JA2IGY |     24 |
|  6 | JI1HFJ     | KH6RS  |     15 |
|  7 | JI1HFJ     | VR2B   |     25 |
|  8 | VU3KAZ     | 4X6TU  |     38 |
|  9 | VU3KAZ     | 5Z4B   |     36 |
| 10 | VU3KAZ     | JA2IGY |     25 |
| 11 | VU3KAZ     | OH2B   |     28 |
| 12 | VU3KAZ     | RR9O   |     29 |
| 13 | VU3KAZ     | VK6RBP |     34 |
| 14 | VU3KAZ     | VR2B   |     37 |
| 15 | VU3KAZ     | ZS6DN  |     27 |


In [187]:
SigReport=DU_Spots_10M[DU_Spots_10M.callsign=='JI1HFJ'].groupby(['dx'])[['db']].max().sort_values(by=['db'],ascending=False).reset_index()

In [188]:
SigReport['KnownPower']=0

SigReport.loc[2, 'KnownPower'] = 400
SigReport.loc[4, 'KnownPower'] = 100
SigReport.loc[6, 'KnownPower'] = 100


SigReport['AntGain']=6

Class=['HP','HP','HP','QRP','LP','QRP','LP','LP','LP','LP']
MaxP=[1200,800,400,5,100,5,100,100,100,100]
SigReport['Class']=Class
#SigReport['MaxP']=MaxP



In [190]:
df_to_md(SigReport)

|    | dx     |   db |   KnownPower |   AntGain | Class   |
|---:|:-------|-----:|-------------:|----------:|:--------|
|  0 | DU3T   |   36 |            0 |         6 | HP      |
|  1 | DU1EV  |   34 |            0 |         6 | HP      |
|  2 | DV3A   |   30 |          400 |         6 | HP      |
|  3 | 4I1EBC |   27 |            0 |         6 | QRP     |
|  4 | 4D3X   |   25 |          100 |         6 | LP      |
|  5 | 4F3OM  |   23 |            0 |         6 | QRP     |
|  6 | 4F3BZ  |   21 |          100 |         6 | LP      |
|  7 | DU1WBX |   13 |            0 |         6 | LP      |
|  8 | DU1VGX |    6 |            0 |         6 | LP      |
|  9 | DU1WN  |    2 |            0 |         6 | LP      |
